## Planejamento da solução (IOT)

### Input - Entrada

1. Problema de negócio
    - Selecionar os clientes mais valiosos para integrar um programa de fidelização
    
    
2. Conjunto de dados
    - Vendas de um e-comerce online, durante o período de um ano

### Output - Saída

1. A indicação das pessoas que farão parte do programa de Insiders
    - Lista: client_id | is_insider
                10254  | yes
                88347  | no
                
2. Relatório com as respostas das pergumtas de negócio

    - Quem são as pessoas elegíveis para participar do programa de Insiders ?
    
    - Quantos clientes farão parte do grupo?
    
    - Quais as principais características desses clientes ?
    
    - Qual a porcentagem de contribuição do faturamento, vinda do Insiders ?
    
    - Qual a expectativa de faturamento desse grupo para os próximos meses ?
    
    - Quais as condições para uma pessoa ser elegível ao Insiders ?
    
    - Quais as condições para uma pessoa ser removida do Insiders ?
    
    - Qual a garantia que o programa Insiders é melhor que o restante da base ?
    
    - Quais ações o time de marketing pode realizar para aumentar o faturamento?

### Tasks - Tarefas

1. Quem são as pessoas elegíveis para participar do programa de Insiders ?
    - O que é ser elegível? O que são clientes de maior "valor"?
    - Faturamento:
        - Alto ticket médio.
        - Alto LTV.
        - Baixa recência.
        - Alto basket size.
        - Baixa probabilidade de churn.
        - Alta previsão LTV.
        - Alta propensão de compra.
        
    - Custo:
        - Baixa taxa de devolução.
        
    - Experiencia de compra:
        - Média alta de avaliação
        

2. Quantos clientes farão parte do grupo?
    - Número total de clientes;
    - % do grupo Insiders
    

3. Quais as principais características desses clientes ?
    - Escrever características do cliente:
        - Idade;
        - Localização;
        - País;
        
    - Escrever caracteristicas do consumo:
        - Atributos da clusterização


4. Qual a porcentagem de contribuição do faturamento, vinda do Insiders ?
    - Faturamento total do ano;
    - Faturamento do grupo Insiders;


5. Qual a expectativa de faturamento desse grupo para os próximos meses ?
     - LTV do grupo Insiders;
     - Análise de Cohort;
     - Séries temporais (ARIMA, HoltWinter);


6. Quais as condições para uma pessoa ser elegível ao Insiders ?
     - Definir a periodicidade (1 mês, 3 mêses);
     - A pessoa precsa ser similar ou parecido com uma pessoa do grupo
     

7. Quais as condições para uma pessoa ser removida do Insiders ?
    - Definir a periodicidade (1 mês, 3 mêses);
    - A pessoa precisa ser desimilar ou não-parecido com uma pessoa do grupo


8. Qual a garantia que o programa Insiders é melhor que o restante da base ?
    - Teste A/B
    - Teste A/B Bayesiano
    - Teste de hipóteses


9. Quais ações o time de marketing pode realizar para aumentar o faturamento?
    - Desconto;
    - Preferencia de compra;
    - Produtos exclusivos

# 0.0 Imports

In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.display import HTML

## 0.1 Helper Functions

In [35]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [18, 9]
    plt.rcParams['font.size'] = 24

    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    pd.set_option('display.float_format', lambda x: '%.2f' % x)
    sns.set()
    
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


## 0.2 Load Data

In [13]:
df_raw = pd.read_csv('data/Ecommerce.csv')

In [14]:
# droping extra column
df_raw.drop(columns=['Unnamed: 8'], axis=1, inplace=True)

# 1.0 Data Description

In [15]:
df1 = df_raw.copy()

## 1.1 Rename columns

In [16]:
df1.columns = ['invoice_no','stock_code','description','quantity','invoice_date','unit_price','customer_id','country']

## 1.2 Data Dimensions

In [17]:
df1.shape

(541909, 8)

## 1.3 Data types

In [18]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

## 1.4 Check Na

In [20]:
df1.isnull().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

## 1.5 Replace Na

In [21]:
# remove na
df1.dropna(subset=['description','customer_id'], inplace=True)

## 1.6 Change Types

In [27]:
# invoice date
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format='%d-%b-%y')

# customer_id
df1['customer_id'] = df1['customer_id'].astype(int)

## 1.7 Descriptive Statistics 

# 2.0 Feature Engineering

In [62]:
df2 = df1.copy()

## 2.1 Feature Creation

In [63]:
df_ref = df2.drop(['invoice_no','stock_code','description','quantity',
                   'invoice_date','unit_price','country'], axis=1).drop_duplicates(ignore_index=True)
df_ref.head()

,customer_id
0,17850
1,13047
2,12583
3,13748
4,15100


In [64]:
# Gross Revenue (Faturamento)
df2['gross_revenue'] = df2['quantity'] * df2['unit_price']

# Monetary
df_monetary = df2[['customer_id','gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge(df_ref, df_monetary, on='customer_id', how='left')

# Recency - last day purchase
df_recency = df2[['customer_id','invoice_date']].groupby('customer_id').max().reset_index()
df_recency['recency_days'] = (df2['invoice_date'].max() - df_recency['invoice_date']).dt.days
df_recency = df_recency[['customer_id','invoice_date']].copy()
df_ref = pd.merge(df_ref, df_recency, on='customer_id', how='left')

# Frequency
df_freq = df2[['customer_id','invoice_no']].drop_duplicates().groupby('customer_id').count().reset_index()
df_ref = pd.merge(df_ref, df_freq, on='customer_id', how='left')

# 3.0 Variable Filtering

# 4.0  EDA

# 5.0 Data Preparation

# 6.0 Feature Selection

# 7.0 Hyperparameter Fine_Tunning

# 8.0 Model Training

# 9.0 Cluster Analysis

# 10.0 Deploy